In [1]:
%pip install wandb datasets transformers trl torch peft huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import wandb
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig
import torch
from peft import LoraConfig, get_peft_model

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
The token `mac` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing

In [4]:
# 모델과 토크나이저 로드
model_name = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# corpus.json 파일 읽기
with open('./data/corpus.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# train/test 분리 (80:20 비율)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"전체 데이터 크기: {len(data)}")
print(f"학습 데이터 크기: {len(train_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")

# 데이터 예시 출력
print("\n=== 학습 데이터 예시 ===")
print(train_data[0])
print("\n=== 테스트 데이터 예시 ===")
print(test_data[0])

# 리스트를 Dataset 형식으로 변환
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

전체 데이터 크기: 108
학습 데이터 크기: 86
테스트 데이터 크기: 22

=== 학습 데이터 예시 ===
{'input': '감정: 외로움 / 원하는 효과: 공감받고 싶다 / 직업: 프리랜서 / 현재 상황: 잠들기 전 휴식 / 집중도: 가볍게 읽을 수 있는 책', 'instruction': '외로움을 공감하며 위로받을 수 있는 3권을 선정, 추천 이유와 효과를 각각 기술하세요.', 'output': '1. 외로움의 습격 - 김만권\n\n* 추천 이유: 외로움의 본질과 현실을 있는 그대로 바라보고, 저자의 통찰과 사례를 통해 공감을 느낄 수 있습니다.\n* 이 책이 도움이 될 수 있는 이유: 외로움이 정상이자 성장의 동기가 될 수 있음을 일깨워줍니다.\n\n2. 혼자가 좋지만 고독사는 걱정입니다 - 몬가 미오코\n\n* 추천 이유: 혼자 사는 삶에 대한 여러 고민을 담아 유연하게 받아들이는 힘을 선사합니다.\n* 이 책이 도움이 될 수 있는 이유: 혼자와 고독에 대한 두려움을 덜고, 자신만의 행복을 느낄 수 있습니다.\n\n3. (쇼펜하우어의) 고독한 행복 - 아르투어 쇼펜하우어\n\n* 추천 이유: 철학자의 시선을 통해 고독을 긍정적인 감정으로 전환하고 싶을 때 적합합니다.\n* 이 책이 도움이 될 수 있는 이유: 철학적 사유와 실질적인 삶의 지혜를 함께 얻을 수 있습니다.'}

=== 테스트 데이터 예시 ===
{'input': '감정: 만족 / 원하는 효과: 새로운 시각과 영감 / 직업: 학생 / 상황: 주말 / 집중도: 깊이 몰입할 수 있는 책', 'instruction': '아래 상황에 맞는 3권을 선정, 추천 이유와 감정적 효과를 포함하여 작성.', 'output': '1. 각본 없음 - 아비 모건\n\n* 추천 이유: 단순한 이야기가 아닌 인생의 우연성과 변화를 직시할 수 있는 힘을 줍니다.\n* 이 책이 도움이 될 수 있는 이유: 자신의 삶을 주체적으로 바라보고 긍정하는 영감의 시간이 됩니다.\n\n2. 공감사회를 위한 담론들 - 강황선\n

In [6]:
# 프롬프트 포매팅 함수 정의
def formatting_func(example):
    # instruction, input, output이 단일 문자열인 경우를 처리
    instruction = example['instruction']
    input_text = example['input'].strip() if example['input'] else ""
    output = example['output']

    # 형식화된 프롬프트 생성
    text = f"[Instruction]\n{instruction.strip()}\n\n"
    if input_text:
        text += f"[Input]\n{input_text}\n\n"
    text += f"[Output]\n{output.strip()}"

    return text

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal Language Modeling을 위해 False로 설정
)

In [ ]:
wandb.init(
    project='Hanghae99-book-recommandation',
    name=f'gpt-finetuning',
    reinit=True
)

model = AutoModelForCausalLM.from_pretrained(model_name)

trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=SFTConfig(
        output_dir=f"/tmp/clm-instruction-tuning",
        max_seq_length=128,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=3,
        learning_rate=2e-5,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss"
    ),
    formatting_func=formatting_func,
    data_collator=data_collator,
)

# 학습 시작
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)

trainer.save_model()

wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: knospe1 (knospe1-gaeun) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### LoRA 적용

In [ ]:
def train_with_lora(lora_r, model_name, train_dataset, test_dataset, formatting_func, data_collator):
    # CUDA 캐시 초기화
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f'초기 가용 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory/1024**3:.1f} GB')

    wandb.init(
        project='Hanghae99-8-basic',
        name=f'gpt-finetuning-with-lora-r{lora_r}',
        reinit=True
    )

    print(f"\n=== Training with LoRA rank {lora_r} ===")

    # 기본 모델 로드 시 메모리 최적화 옵션 추가
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map='auto',  # 자동으로 최적의 디바이스 할당
        torch_dtype=torch.float16  # 16비트 부동소수점 사용으로 메모리 절약
    )

    # LoRA 설정
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    # LoRA 적용
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    trainer = SFTTrainer(
        model,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        args=SFTConfig(
            output_dir=f"/tmp/clm-instruction-tuning-lora-{lora_r}",
            max_seq_length=128,
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_steps=100,
            num_train_epochs=3,
            learning_rate=2e-5,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            per_device_train_batch_size=4,  # 배치 사이즈 조정
            gradient_accumulation_steps=4,  # 그래디언트 누적으로 메모리 사용량 감소
            fp16=True  # 16비트 학습 활성화
        ),
        formatting_func=formatting_func,
        data_collator=data_collator,
    )

    # 학습 시작
    train_result = trainer.train()
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)

    # 메모리 사용량 출력
    if torch.cuda.is_available():
        print('Max Alloc:', round(torch.cuda.max_memory_allocated(0)/1024**3, 1), 'GB')

    # 모델 저장
    trainer.save_model()

    # CUDA 캐시 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    wandb.finish()

In [ ]:
train_with_lora(
    lora_r=8,
    model_name=model_name,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    formatting_func=formatting_func,
    data_collator=data_collator
)